# Assignment 2: IR

## Preparations
* Put all your imports, and path constants in the next cells
* Make sure all your path constants are **relative to** ***DATA_DIR*** and **NOT hard-coded** in your code.

In [1]:
!pip install whoosh
!pip install pytrec_eval
!pip install wget

In [2]:
import wget
wget.download("https://github.com/MIE451-1513-2019/course-datasets/raw/master/government.zip", "government.zip")

'government (1).zip'

In [3]:
!unzip government.zip

Archive:  government.zip
replace government/topics-with-full-descriptions.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace government/topics-with-full-descriptions.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: government/topics-with-full-descriptions.txt  
  inflating: government/gov.topics   
  inflating: government/gov.qrels    
  inflating: government/documents/61/G00-61-2800209  
  inflating: government/documents/61/G00-61-1192048  
  inflating: government/documents/61/G00-61-1118212  
  inflating: government/documents/61/G00-61-0749882  
  inflating: government/documents/61/G00-61-2230501  
  inflating: government/documents/61/G00-61-0680698  
  inflating: government/documents/61/G00-61-0551387  
  inflating: government/documents/61/G00-61-2575433  
  inflating: government/documents/61/G00-61-0469713  
  inflating: government/documents/61/G00-61-0280746  
  inflating: government/documents/61/G00-61-2574316  
  inflating: government/docu

In [0]:
# imports
# Put all your imports here
import numpy as np
import pandas as pd
from whoosh import index, writing
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import *
from whoosh.analysis import Filter
from whoosh.qparser import QueryParser
from whoosh import scoring
from whoosh import qparser
import os.path
from pathlib import Path
import tempfile
import subprocess
import pytrec_eval
import wget
import nltk
from nltk.stem import *


In [0]:
DATA_DIR = "government"
#
# Put other path constants here
#
DOCUMENTS_DIR = os.path.join(DATA_DIR, "documents")
TOPIC_FILE = os.path.join(DATA_DIR, "gov.topics")
QRELS_FILE = os.path.join(DATA_DIR, "gov.qrels")

## Question 1
Provide your text answers in the following two markdown cells

### Q1 (a): Provide answer to Q1 (a) here [markdown cell]

Mean Average Precision (MAP)

### Q1 (b): Provide answer to Q1 (b) here [markdown cell]

MAP takes into consideration average precisions of multiple queries to the search system, thus is able to provide a macro-view of the system's performance

## Question 2

### Q2 (a): Write your code below

In [0]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q2, your query parser in QP_Q2, and your searcher in SEARCHER_Q2

In [0]:
def createIndex(schema):
    # Generate a temporary directory for the index
    indexDir = tempfile.mkdtemp()

    # create and return the index
    return index.create_in(indexDir, schema)

In [0]:
# first, define a Schema for the index
mySchema = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = RegexTokenizer()))

# now, create the index at the path INDEX_DIR based on the new schema
myIndex = createIndex(mySchema)

In [0]:
def addFilesToIndex(indexObj, fileList):
    # open writer
    writer = writing.BufferedWriter(indexObj, period=None, limit=1000)

    try:
        # write each file to index
        for docNum, filePath in enumerate(fileList):
            with open(filePath, "r", encoding="utf-8") as f:
                fileContent = f.read()
                writer.add_document(file_path = filePath,
                                    file_content = fileContent)

                # print status every 1000 documents
                if (docNum+1) % 1000 == 0:
                    print("already indexed:", docNum+1)
        print("done indexing.")

    finally:
        # close the index
        writer.close()

In [0]:
# Build a list of files to index
filesToIndex = [str(filePath) for filePath in Path(DOCUMENTS_DIR).glob("**/*") if filePath.is_file()]

In [11]:
# count files to index
print("number of files:", len(filesToIndex))

number of files: 4078


In [12]:
addFilesToIndex(myIndex, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [0]:
# define a query parser for the field "file_content" in the index
myQueryParser = QueryParser("file_content", schema=myIndex.schema)
mySearcher = myIndex.searcher()

In [14]:
# run a sample query for the phrase "item"
sampleQuery = myQueryParser.parse("physical therapists")
sampleQueryResults = mySearcher.search(sampleQuery, limit=None)

# inspect the result:
# for each document print the rank and the score
for (docnum, result) in enumerate(sampleQueryResults):
    score = sampleQueryResults.score(docnum)
    fileName = os.path.basename(result["file_path"])
    print(fileName, docnum, score)

G00-26-3134051 0 13.996502415475085
G00-59-0786269 1 13.8539343648808
G00-60-3914816 2 11.345260421579761
G00-21-0649032 3 5.95590290335526
G00-45-4032177 4 5.937136717292031


In [0]:
INDEX_Q2 = myIndex # Index for Q2
QP_Q2 = myQueryParser # Query parser for Q2
SEARCHER_Q2 = mySearcher # Searcher for Q2

In [16]:
# print the topic file
with open(TOPIC_FILE, "r") as f:
    print(f.read())

1 mining gold silver coal
2 juvenile delinquency
4 wireless communications
6 physical therapists
7 cotton industry
9 genealogy searches
10 Physical Fitness
14 Agricultural biotechnology
16 Emergency and disaster preparedness assistance
18 Shipwrecks
19 Cybercrime, internet fraud, and cyber fraud
22 Veteran's Benefits
24 Air Bag Safety
26 Nuclear power plants
28 Early Childhood Education



In [17]:
# print the first 10 lines in the qrels file
with open(QRELS_FILE, "r") as f:
    qrels10 = f.readlines()[:10]
    print("".join(qrels10))

1 0 G00-00-0681214 0
1 0 G00-00-0945765 0
1 0 G00-00-1006224 1
1 0 G00-00-1591495 0
1 0 G00-00-2764912 0
1 0 G00-00-3253540 0
1 0 G00-00-3717374 0
1 0 G00-01-0270065 0
1 0 G00-01-0400712 0
1 0 G00-01-0682299 0



In [0]:
def pyTrecEval(topicFile, qrelsFile, queryParser, searcher):
    # Load topic file - a list of topics(search phrases) used for evalutation
    with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()

    # create an output file to which we'll write our results
    tempOutputFile = tempfile.mkstemp()[1]
    with open(tempOutputFile, "w") as outputTRECFile:
        # for each evaluated topic:
        # build a query and record the results in the file in TREC_EVAL format
        for topic in topics:
            topic_id, topic_phrase = tuple(topic.split(" ", 1))
            #print(topic_id, topic_phrase)
            topicQuery = queryParser.parse(topic_phrase)
            topicResults = searcher.search(topicQuery, limit=None)
            for (docnum, result) in enumerate(topicResults):
                score = topicResults.score(docnum)
                #print("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
                outputTRECFile.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
    with open(qrelsFile, 'r') as f_qrel:
        qrel = pytrec_eval.parse_qrel(f_qrel)

    with open(tempOutputFile, 'r') as f_run:
        run = pytrec_eval.parse_run(f_run)

    evaluator = pytrec_eval.RelevanceEvaluator(
        qrel, pytrec_eval.supported_measures)

    results = evaluator.evaluate(run)
    def print_line(measure, scope, value):
        print('{:25s}{:8s}{:.4f}'.format(measure, scope, value))

    for query_id, query_measures in results.items():
        for measure, value in query_measures.items():
            if measure == "runid":
              continue
            print_line(measure, query_id, value)
    for measure in query_measures.keys():
        if measure == "runid":
              continue
        print_line(
            measure,
            'all',
            pytrec_eval.compute_aggregated_measure(
                measure,
                [query_measures[measure]
                 for query_measures in results.values()]))

In [19]:
pyTrecEval(TOPIC_FILE, QRELS_FILE, myQueryParser, mySearcher) 

num_q                    1       1.0000
num_ret                  1       1.0000
num_rel                  1       5.0000
num_rel_ret              1       0.0000
map                      1       0.0000
gm_map                   1       -11.5129
Rprec                    1       0.0000
bpref                    1       0.0000
recip_rank               1       0.0000
iprec_at_recall_0.00     1       0.0000
iprec_at_recall_0.10     1       0.0000
iprec_at_recall_0.20     1       0.0000
iprec_at_recall_0.30     1       0.0000
iprec_at_recall_0.40     1       0.0000
iprec_at_recall_0.50     1       0.0000
iprec_at_recall_0.60     1       0.0000
iprec_at_recall_0.70     1       0.0000
iprec_at_recall_0.80     1       0.0000
iprec_at_recall_0.90     1       0.0000
iprec_at_recall_1.00     1       0.0000
P_5                      1       0.0000
P_10                     1       0.0000
P_15                     1       0.0000
P_20                     1       0.0000
P_30                     1       0.000

In [0]:
def printRelName(topicFile, qrelsFile, queryParser, searcher, id):
  with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()
  for topic in topics:
        topic_id, topic_phrase = tuple(topic.split(" ", 1))
        if topic_id == id:
          print("---------------------------Topic_id and Topic_phrase----------------------------------")
          print(topic_id, topic_phrase)
          topicQuery = queryParser.parse(topic_phrase)
          topicResults = searcher.search(topicQuery, limit=None)
          print("---------------------------Return documents----------------------------------")
          for (docnum, result) in enumerate(topicResults):
              score = topicResults.score(docnum)
              print("%s Q0 %s %d %lf test" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
          print("---------------------------Relevant documents----------------------------------")
          with open(qrelsFile, 'r') as f_qrel:
            qrels = f_qrel.readlines()
            for i in qrels:
              qid, _, doc, rel = i.rstrip().split(" ")
              if qid == id and rel == "1":
                print(i.rstrip())

### Q2 (b): Provide answer to Q2 (b) here [markdown cell]

From the above trec_evaluation, the baseline Whoosh system has a MAP value of 0.1971 over all queries (topics).

### Q2 (c): Provide answer to Q2(c) here [markdown cell]

The MAP for each topic, which is the same as the corresponding AP, is particulardly bad (value of zero) for topics 1, 2, 6, 7, 9, 16, and 28. Conversely, topics 18 and 24 did very well, with an AP value of one each. 

## Question 3

### Q3 (a): Provide answer to Q3 (a) here [markdown cell]

In [21]:
printRelName(TOPIC_FILE, QRELS_FILE, myQueryParser, mySearcher, "14")

---------------------------Topic_id and Topic_phrase----------------------------------
14 Agricultural biotechnology
---------------------------Return documents----------------------------------
14 Q0 G00-79-4144643 0 16.017277 test
14 Q0 G00-09-1193469 1 15.942973 test
14 Q0 G00-45-0809730 2 15.640293 test
14 Q0 G00-89-0000000 3 15.377247 test
14 Q0 G00-88-1894712 4 8.188481 test
14 Q0 G00-34-0444524 5 3.239981 test
14 Q0 G00-84-0274223 6 1.544029 test
---------------------------Relevant documents----------------------------------
14 0 G00-89-0000000 1


Query: #14 Agricultural biotechnology

False positive: Doc # G00-79-4144643

False negative: Doc # G00-89-0000000

In this query case, the false positive document was ranked number one in the baseline system, but is non-relevant according to the printRelname function. The word "Agricultrual" appears in this document 14 times, 12 of which are capitalized in the first letter. The word "biotechnology" appears in the document 5 times, 2 of which are in lower case. 

In false negative document, the word "Agricultrual" appears in this document 4 times, all having upper case in the first letter. The word "biotechnology" appears 10 times, but only one is in lower case. Since the baseline system does not have a lowercase filter, the system will not pick up on the occurances that are not in the same capitalization, dispite being technically relevant. 

Suggestions to improve Whoosh's performance would be to add filters onto the existing tokenizer, in order for the system in attempt to better evaluate the relevance between queries and the documents. These filters include lowercase, stop words, instraword, and stemming. We can also try incorporating stemmers/lemmatizers into the Whoosh system as custom filters.

### Q3 (b): Write your code below

In [0]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q3, your query parser in QP_Q3, and your searcher in SEARCHER_Q3

In [0]:
#built-in filters
stmLwrStpIntraAnalyzer = RegexTokenizer() | LowercaseFilter() | StopFilter() | IntraWordFilter() | StemFilter() 

In [0]:
# Dont change this! Use it as-is in your code
# This filter will run for both the index and the query
class CustomFilter(Filter):
    is_morph = True
    def __init__(self, filterFunc, *args, **kwargs):
        self.customFilter = filterFunc
        self.args = args
        self.kwargs = kwargs
    def __eq__(self):
        return (other
                and self.__class__ is other.__class__)
    def __call__(self, tokens):
        for t in tokens:
            if t.mode == 'query': # if called by query parser
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t
            else: # == 'index' if called by indexer
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t

In [25]:
#custom NLTK stemmer/lemmatizer filters
nltk.download("wordnet")
myFilter1 = RegexTokenizer() | CustomFilter(LancasterStemmer().stem)
myFilter2 = RegexTokenizer() | CustomFilter(WordNetLemmatizer().lemmatize)
myFilter3 = RegexTokenizer() | CustomFilter(WordNetLemmatizer().lemmatize, 'v')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
# combinations of built-in filters and NLTK stemmers and lemmatizers
Filters_Combined_1 = RegexTokenizer() | LowercaseFilter() | StopFilter() | IntraWordFilter() | StemFilter() | CustomFilter(LancasterStemmer().stem)
Filters_Combined_2 = RegexTokenizer() | LowercaseFilter() | StopFilter() | IntraWordFilter() | StemFilter() | CustomFilter(WordNetLemmatizer().lemmatize)
Filters_Combined_3 = RegexTokenizer() | LowercaseFilter() | StopFilter() | IntraWordFilter() | StemFilter() | CustomFilter(WordNetLemmatizer().lemmatize, 'v')

In [0]:
# define a Schema with the new analyzer
mySchema2 = Schema(file_path = ID(stored=True),
                   file_content = TEXT(analyzer = Filters_Combined_1))

# create the index based on the new schema
myIndex2 = createIndex(mySchema2)

In [28]:
addFilesToIndex(myIndex2, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [0]:
# define a query parser for the field "file_content" in the index
myQueryParser2 = QueryParser("file_content", schema=myIndex2.schema)
mySearcher2 = myIndex2.searcher()

In [30]:
pyTrecEval(TOPIC_FILE, QRELS_FILE, myQueryParser2, mySearcher2) 

num_q                    1       1.0000
num_ret                  1       3.0000
num_rel                  1       5.0000
num_rel_ret              1       0.0000
map                      1       0.0000
gm_map                   1       -11.5129
Rprec                    1       0.0000
bpref                    1       0.0000
recip_rank               1       0.0000
iprec_at_recall_0.00     1       0.0000
iprec_at_recall_0.10     1       0.0000
iprec_at_recall_0.20     1       0.0000
iprec_at_recall_0.30     1       0.0000
iprec_at_recall_0.40     1       0.0000
iprec_at_recall_0.50     1       0.0000
iprec_at_recall_0.60     1       0.0000
iprec_at_recall_0.70     1       0.0000
iprec_at_recall_0.80     1       0.0000
iprec_at_recall_0.90     1       0.0000
iprec_at_recall_1.00     1       0.0000
P_5                      1       0.0000
P_10                     1       0.0000
P_15                     1       0.0000
P_20                     1       0.0000
P_30                     1       0.000

In [31]:
printRelName(TOPIC_FILE, QRELS_FILE, myQueryParser2, mySearcher2, "14")

---------------------------Topic_id and Topic_phrase----------------------------------
14 Agricultural biotechnology
---------------------------Return documents----------------------------------
14 Q0 G00-89-0000000 0 18.223529 test
14 Q0 G00-79-4144643 1 16.296629 test
14 Q0 G00-97-1475424 2 15.719473 test
14 Q0 G00-01-3251318 3 15.719473 test
14 Q0 G00-51-1924264 4 15.496427 test
14 Q0 G00-72-0385489 5 15.016486 test
14 Q0 G00-86-2926152 6 14.190321 test
14 Q0 G00-45-0809730 7 14.084022 test
14 Q0 G00-09-1193469 8 13.922232 test
14 Q0 G00-36-2788975 9 13.568149 test
14 Q0 G00-97-2215955 10 13.463391 test
14 Q0 G00-09-1404763 11 13.247946 test
14 Q0 G00-10-2024294 12 13.191933 test
14 Q0 G00-91-1609512 13 13.180014 test
14 Q0 G00-06-2853218 14 12.543855 test
14 Q0 G00-35-2527252 15 12.486037 test
14 Q0 G00-06-0690672 16 12.132965 test
14 Q0 G00-86-0847220 17 11.497796 test
14 Q0 G00-88-1894712 18 11.298202 test
14 Q0 G00-70-3424520 19 10.934581 test
14 Q0 G00-07-2371962 20 9.176561 te

In [0]:
INDEX_Q3 = myIndex2 # Replace None with your index for Q3
QP_Q3 = myQueryParser2 # Replace None with your query parser for Q3
SEARCHER_Q3 = mySearcher2 # Replace None with your searcher for Q3

### Q3 (c): Provide answer to Q3 (c) here [markdown cell]

The filters lowercase, stop-word, stemp, intraword, and lancaster stemmer are added to the Whoosh system, resulting in a MAP value of 0.3568. 

For the query #14 as discussed previously, the false negative/positive documents have been improved, with the single relevant document being ranked first by the system. 

### Q3 (d): Provide answer to Q3 (d) here [markdown cell]

yes

### Q3 (e): Provide answer to Q3 (e) here [markdown cell]

yes

### Q3 (f): Provide answer to Q3 (f) here [markdown cell]

Individually, performance of the system on 5 topics (#2, #4, #10, #14, and #28) were improved, while performance on 2 topics (#22 and #26) got worse. However, the overall performance of the system as reflected by the MAP value was improved from 0.1971 of the baseline to 0.3568 of the new system (an 81% improvement). Therefore, this new system with the addition of filters has resulted in improvements over the baseline system. 

## Question 4

In [0]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q4, your query parser in QP_Q4, and your searcher in SEARCHER_Q4

In [0]:
# define a Schema with the new analyzer
mySchema3 = Schema(file_path = ID(stored=True),
                   file_content = TEXT(analyzer = Filters_Combined_1))

# create the index based on the new schema
myIndex3 = createIndex(mySchema3)

In [35]:
addFilesToIndex(myIndex3, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [0]:
# this function only prints the overall map value 
def pyTrecEvalMAP(topicFile, qrelsFile, queryParser, searcher):
    # Load topic file - a list of topics(search phrases) used for evalutation
    with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()

    # create an output file to which we'll write our results
    tempOutputFile = tempfile.mkstemp()[1]
    with open(tempOutputFile, "w") as outputTRECFile:
        # for each evaluated topic:
        # build a query and record the results in the file in TREC_EVAL format
        for topic in topics:
            topic_id, topic_phrase = tuple(topic.split(" ", 1))
            #print(topic_id, topic_phrase)
            topicQuery = queryParser.parse(topic_phrase)
            topicResults = searcher.search(topicQuery, limit=None)
            for (docnum, result) in enumerate(topicResults):
                score = topicResults.score(docnum)
                #print("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
                outputTRECFile.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
    with open(qrelsFile, 'r') as f_qrel:
        qrel = pytrec_eval.parse_qrel(f_qrel)

    with open(tempOutputFile, 'r') as f_run:
        run = pytrec_eval.parse_run(f_run)

    evaluator = pytrec_eval.RelevanceEvaluator(
        qrel, pytrec_eval.supported_measures)

    results = evaluator.evaluate(run)
    def print_line(value):
        print('{:.4f}'.format(value))

    for query_id, query_measures in results.items():
        for measure, value in query_measures.items():
            if measure == "map" and query_id == "all":
              print_line(value)
    for measure in query_measures.keys():
        if measure == "map":
          print_line(
            pytrec_eval.compute_aggregated_measure(
                measure,
                [query_measures[measure]
                 for query_measures in results.values()]))

In [37]:
# setting up parser and searcher(scoring)
# using orgroup for parsing queries
og = qparser.OrGroup
myQueryParser3 = QueryParser("file_content", schema=myIndex3.schema, group=og)

# using BM25F ranking with parameters of B and k1 being free parameters that needs to be optimized

# maximum value is obtained at i=5
for i in range (0,6):
    print('i =', i)
    mySearcher3 = myIndex3.searcher(weighting=scoring.BM25F(B=i*0.1, k1=i))
    pyTrecEvalMAP(TOPIC_FILE, QRELS_FILE, myQueryParser3, mySearcher3)

i = 0
0.2686
i = 1
0.3319
i = 2
0.3381
i = 3
0.3811
i = 4
0.3999
i = 5
0.4028


In [38]:
#value for measure = 'map', scope = 'all'
#map is at maximum at 0.3883 for free parameters B = 0.5 and k1 = 5
mySearcher3 = myIndex3.searcher(weighting=scoring.BM25F(B=0.5, k1=5))
pyTrecEvalMAP(TOPIC_FILE, QRELS_FILE, myQueryParser3, mySearcher3) 

0.4028


### Please answer the following questions here
(a) A clear list of all final modifications made.  

1. Filters used in the analyzer: lowercase, stop-word, stemp, intraword, and lancaster stemmer

2. Setting scoring parameters B = 0.5 and k1 = 5 for the BM25F scoring system.

3. Using "orGroup" for the query parser


(b)  Why each modification was made – how did it help?  

  Reasonings for each modification made above: 

1. The filters used in Q4 are the same as the ones used in Q3, since they significantly improved the performance of the system.

2. The default parser joins tokens using "AND" and therefore requires all tokens to be present in order for the document to match. However, by using the "orgroup" option, the tokens are treated as either/or, meaning a document can match if any token is present. This would make it less likely that the system incorrectly identifies a document as less relevant. 

3. Although by default the searcher uses the B25F scoring algorithm, the parameters k1 and B is not optimized for our system. A simple loop optimization is used to calculate the ideal values for k1 and B to obtain a maximum MAP value of the system. 

(c)  The  final  MAP  performance  that  these  modifications  attained.

The final MAP value found is 0.4028.

In [0]:
INDEX_Q4 = myIndex3 # Replace None with your index for Q4
QP_Q4 = myQueryParser3 # Replace None with your query parser for Q4
SEARCHER_Q4 = mySearcher3 # Replace None with your searcher for Q4

## Validation

In [0]:
# Run the following cells to make sure your code returns the correct value types

In [0]:
from whoosh.index import FileIndex
from whoosh.qparser import QueryParser
from whoosh.searching import Searcher
import os.path

### Q2 Validation

In [42]:
assert(isinstance(INDEX_Q2, FileIndex)), "Index Type"
assert(isinstance(QP_Q2, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q2, Searcher)), "Searcher Type"
print("Q2 Types Validated")

Q2 Types Validated


### Q3 Validation

In [43]:
assert(isinstance(INDEX_Q3, FileIndex)), "Index Type"
assert(isinstance(QP_Q3, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q3, Searcher)), "Searcher Type"
print("Q3 Types Validated")

Q3 Types Validated


### Q4 Validation

In [44]:
assert(isinstance(INDEX_Q4, FileIndex)), "Index Type"
assert(isinstance(QP_Q4, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q4, Searcher)), "Searcher Type"
print("Q4 Types Validated")

Q4 Types Validated
